# Lab 3, part II: solving problems with a subset of columns

This lab focuses on solving an LP considering a subset of its variables. This is useful when the number of variables is very large as one can still solve the LP to optimality.

Consider again the product mix problem: a factory has recipes for producing $n=40$ types of perfume by mixing 5 ingredients. Denote as $P_0=\{0,1,2,\ldots{},39\}$ the set of 40 perfumes, and as $I=\{0,1,2,3,4\}$ the set of ingredients. The problem is defined as in part I, with the exception that $A$ and $c$ have more elements. Their value is specified directly in the code snippet after this cell. The first four elements of $c$ and the first four columns of $A$ are the same as in the previous exercise. The availability is the same as in the previous part:

|Ingredient|0|1|2|3|4|
|----------|-|-|-|-|-|
|Availability|30|400|90|450|70|

The following are the tasks for this exercise:

1. Solve the problem with the first four perfumes only, i.e., create and solve the problem with the same data as that of part I;
2. Suppose now you could add __one__ perfume to the first four; which one should be added to increase (even by a tiny amount) the objective function?
3. Repeat task 2 after adding the chosen column; which one should be the sixth variable?
4. Solve the problem with __all__ $n$ columns and report the new revenue; before solving it, can you predict if it will be larger or smaller? Why?

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [ ]:
import numpy as np  # Numpy is handy for vector&matrix operations

A = \
np.array([[0.01, 0.05, 0.07, 0.04, 0.07, 0.17, 0.08, 0.14, 0.03, 0.01, 0.03, 0.03, 0.06, 0.06, 0.02, 0.05, 0.04, 0.01, 0.03, 0.04, 0.01, 0.02, 0.01, 0.02, 0.04, 0.02, 0.02, 0.11, 0.03, 0.02, 0.02, 0.03, 0.04, 0.06, 0.05, 0.04, 0.01, 0.05, 0.06, 0.04],
          [0.34, 0.45, 0.36, 0.51, 0.41, 0.15, 0.68, 0.39, 0.65, 0.27, 0.25, 0.11, 0.03, 0.69, 0.5 , 0.04, 0.33, 0.69, 0.32, 0.46, 0.53, 0.03, 0.39, 0.29, 0.68, 0.26, 0.53, 0.14, 0.3 , 0.76, 0.49, 0.49, 0.46, 0.15, 0.43, 0.27, 0.58, 0.53, 0.23, 0.63],
          [0.08, 0.06, 0.12, 0.12, 0.14, 0.04, 0.12, 0.05, 0.14, 0.14, 0.11, 0.15, 0.11, 0.19, 0.02, 0.17, 0.1 , 0.04, 0.1 , 0.04, 0.13, 0.13, 0.07, 0.01, 0.14, 0.14, 0.11, 0.2 , 0.12, 0.07, 0.14, 0.11, 0.09, 0.19, 0.15, 0.05, 0.24, 0.06, 0.16, 0.15],
          [0.55, 0.35, 0.29, 0.32, 0.36, 0.49, 0.12, 0.09, 0.12, 0.51, 0.56, 0.59, 0.79, 0.06, 0.33, 0.68, 0.48, 0.26, 0.55, 0.4 , 0.31, 0.75, 0.53, 0.66, 0.1 , 0.56, 0.31, 0.34, 0.55, 0.05, 0.33, 0.33, 0.41, 0.58, 0.31, 0.64, 0.01, 0.3 , 0.51, 0.08],
          [0.02, 0.09, 0.16, 0.01, 0.02, 0.15, 0.  , 0.33, 0.06, 0.07, 0.05, 0.12, 0.01, 0.  , 0.13, 0.06, 0.05, 0.  , 0.  , 0.06, 0.02, 0.07, 0.  , 0.02, 0.04, 0.02, 0.03, 0.21, 0.  , 0.1 , 0.02, 0.04, 0.  , 0.02, 0.06, 0.  , 0.16, 0.06, 0.04, 0.1 ]])

b = np.array([30, 400, 90, 450, 70])

c = np.array([300, 255, 260, 390, 243, 93, 310, 117, 286, 267,
              250, 276, 324, 262, 216, 310, 285, 266, 280, 190,
              241, 301, 271, 252, 326, 279, 310, 278, 250, 248,
              319, 281, 287, 279, 299, 274, 305, 209, 250, 296])

# Verify shape of all data
print(f"A has size {A.shape}, b has size {b.shape}, c has size {c.shape}")

## Solution

Task 1: __Solve the problem with the first four perfumes only, i.e., create a problem with the same data as that of part I.__

We have the problem from part I, let's repeat it here:

$$
\begin{array}{lrrrrrr}
\max          & 300 x_0 &+  255 x_1 &+  260 x_2 &+  390 x_3\\
\textrm{s.t.} &0.01 x_0 &+ 0.05 x_1 &+ 0.07 x_2 &+ 0.04 x_3 &\le & 30\\
              &0.34 x_0 &+ 0.45 x_1 &+ 0.36 x_2 &+ 0.51 x_3 &\le &400\\
              &0.08 x_0 &+ 0.06 x_1 &+ 0.12 x_2 &+ 0.12 x_3 &\le & 90\\
              &0.55 x_0 &+ 0.35 x_1 &+ 0.29 x_2 &+ 0.32 x_3 &\le &450\\
              &0.02 x_0 &+ 0.09 x_1 &+ 0.16 x_2 &+ 0.01 x_3 &\le & 70\\
              &x_0, &x_1, &x_2, &x_3 &\ge& 0,
\end{array}
$$

The LPs we'll solve in this part may have different subsets of columns, so we define a function that is similar to part I, but it also takes a subset of indices as an extra argument.

In [ ]:
import mip

I = range(5)  # set of ingredients

def solve_productmix_indices(A, b, c, indices):
    """
    Solve problem max{cx: Ax <= b, x >= 0} but only consider columns
    as indicated by the list "indices"
    """
    m = mip.Model()

    P = # create list of perfumes filtered by "indices"

    x = # (TODO)

    # One constraint per ingredient (TODO)
    for i in I:
        m.add_constr(...)

    # Objective function is a weighted sum of all x (TODO)
    m.objective = mip.maximize(...)
    # Solve
    m.optimize()

    # (TODO) Return a tuple containing model, solution variables, and objective value
    return (m, [], m.objective_value)

# In order to solve the same problem as part I, we pass the first
# four indices as the last argument to the new function
_, solution, objective = solve_productmix_indices(A, b, c, [0,1,2,3])

print("Solution:", solution)
print("Objective: {0:10.2f}".format(objective))

Task 2: __Suppose now you could add _one_ perfume to the first four; which one should be added to increase the objective function?__

An idea could be to run a loop where we solve the above problem with indices `[0,1,2,3,p]` for every $p \in {4,...,39}$, and then report the increase in objective function, if any. Can there be a decrease? Will any new perfume trigger an increase in $x_2$, i.e., `solution[2]`?

In [ ]:
for p in range(4,40):  # uses all p from 4 to 39
    _, sol2, obj2 = solve_productmix_indices() # (TODO) solve problem with indices 0,1,2,3,p
    rndsol2 = [round(s, ndigits=1) for s in sol2]
    print(f"Added perfume {p:2d} --> {obj2:10.2f} (diff: {obj2 - objective:8.2f}). Sol: {rndsol2}")

There are four perfumes that are worth introducing in order to increase revenue. A few remarks:

* When they are produced, perfume 1 is no longer produced. Why? And how much should we increase its cost `c[1]` in order to resume its production (at the expense of another perfume)?
* Perfume 3 always increases production with the introduction of a new perfume with nonzero production.
* This is not true for perfume 0: sometimes its production increases, sometimes it decreases.

Again, solving 36 LPs to find out the difference was not necessary. From your course lectures, you can immediately get this information using the dual variables for each of the resource constraints. This could be done by computing the reduced costs of all variables that were not introduced yet: consider perfume $p\in 0,\ldots{},39$. If $\eta_i$ is the dual variable of constraint $i$, the dual constraint for each primal variable $x_p$ is
$\sum_{i\in I} a_{ip} \eta_i \ge c_p$, or

$$
    a_{0p} \eta_0 + a_{1p} \eta_1 + a_{2p} \eta_2 + a_{3p} \eta_3 + a_{4p} \eta_4 \ge c_p
$$

After solving the initial problem and obtaining optimal values $x^*_0, x^*_1, x^*_2, x^*_3$, we also obtain values $\eta^*_0,\eta^*_1,\ldots, \eta^*_4$ for the dual variables. If we consider a perfume $p$ that has not been added to the problem yet, we can use the dual solution to compute

$$
rc_p = c_p - (a_{0p} \eta^*_0 + a_{1p} \eta^*_1 + a_{2p} \eta^*_2 + a_{3p} \eta^*_3 + a_{4p} \eta^*_4)
$$

This value is the reduced cost of perfume $p$, whether or not the perfume was considered in the problem. If $rc_p \ge 0$, the perfume does not need to be added to the mix, otherwise it will be added. This allows us to determine in advance what perfumes can potentially increase the revenue. Let's compute these reduced costs in a loop:

In [ ]:
# Solve the problem again from scratch, just to make sure 
# we didn't lose solution/objective on the way.
m, solution, objective = solve_productmix_indices (A, b, c, [0, 1, 2, 3])

dual = [con.pi for con in m.constrs]

for p in range(4, 40):  # For ALL perfumes

    # compute reduced cost of all variables, including
    # those not yet in the problem (TODO)
    rc = 
    
    print(f"Reduced cost for {p:2d} = {rc:9.2f}", end='')
    if rc > 0:
        print(" <======")
    else:
        print("")

The reduced costs are __not__, in general, proportional to the change in revenue: perfume 35 has a smaller reduced cost than 23 but a higher revenue change.

Task 3: __Repeat step 2 after adding the chosen column; which one should be the sixth variable?__

Let's choose perfume 14 and re-run the previous cell to determine if there are perfumes to be added. Obviously after solving the problem the reduced cost for perfume 14 will be nonnegative. But which one: zero or positive?

In [ ]:
# Replace starting solution by solving a problem with perfume 14 added (TODO)

print(f"New objective: {objective}; solution = {solution}")

dual = [c.pi for c in m.constrs]

for p in range(4, 40):  # For ALL perfumes
    # Find reduced cost by same computation above (TODO)
    rc = ...
    print(f"Reduced cost for {p:2d} = {rc:9.2f}", end='')
    if rc > 0:
        print(" <======")
    else:
        print("")

Task 4: __Solve the problem with all $n$ columns and report the new revenue; before solving it, can you predict if it will be larger or smaller? Why?__

In [ ]:
# Solve problem with all columns this time (TODO)

print(f"With all perfumes, objective: {objective}")
print(f"Solution:")
for p in range(40):
    if solution[p] > 0:
        print(f"{p:3d}: {solution[p]:10.2f}")

Can we automate this?